In [3]:
import numpy as np
import pandas as pd
from scipy.stats import mode
from scipy.spatial import distance
 
class knn_hand:
    def __init__(self, general_data, k=1):
        self.k = k
        self.data = general_data

    # general data = {classes:data}
    def predict(self, target):

        distances = []

        if any(isinstance(el, list) for el in target):
            winners = []
            for i in target:
                for key in self.data:
                    for data in self.data[key]:
                        for position in data:
                            distances.append([key, distance.euclidean(np.array(position),
                                                                      np.array(i))])
                distances = sorted(distances, key=lambda x:x[1])[:self.k]

                keys = [i[0] for i in distances]

                winners.append(mode(keys)[0][0])
            return winners

        else:
            for key in self.data:
                for data in self.data[key]:
                    for position in data:
                        distances.append([key, distance.euclidean(np.array(position),
                                                                np.array(target))])
            
            distances = sorted(distances, key=lambda x:x[1])[:self.k]

            keys = [i[0] for i in distances]

            winner = mode(keys)[0][0]

            return winner

In [4]:
df = pd.read_csv("/home/rootsyl/Stat/datasets/Heartdesease/heart.csv")
df = df.sample(frac=1)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
179,57,1,0,150,276,0,0,112,1,0.6,1,1,1,0
256,58,1,0,128,259,0,0,130,1,3.0,1,2,3,0
281,52,1,0,128,204,1,1,156,1,1.0,1,0,0,0
121,59,1,0,138,271,0,0,182,0,0.0,2,0,2,1
166,67,1,0,120,229,0,0,129,1,2.6,1,2,3,0


In [5]:
df_general = df.iloc[:round(len(df)*.8)]
df_test = df.iloc[round(len(df)*.8):]
df_general = df_general.groupby('target')[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal']].apply(lambda g: g.values.tolist()).to_dict()

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
model_hand = knn_hand(df_general, k=10)
model_sk = KNeighborsClassifier(n_neighbors=10, weights='distance')
model_sk.fit(df.drop("target", axis=1), df['target'])

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [7]:
df_test_x = df_test.drop("target", axis=1).values.tolist()
df_test_y = df_test['target']

In [8]:
accuracy_score(model_sk.predict(df_test_x), df_test_y)


1.0

In [9]:
accuracy_score(model_hand.predict(df_test_x), df_test_y)

0.5901639344262295

In [10]:
confusion_matrix(model_hand.predict(df_test_x), df_test_y)

array([[ 2,  0],
       [25, 34]])

In [11]:
confusion_matrix(model_sk.predict(df_test_x), df_test_y)

array([[27,  0],
       [ 0, 34]])